## C S 329E HW 5

# Crossvalidation and hyperparameter selection

## Jeremy Ulfohn // A5 Group 3 // jau392
## Anthony Roth-Giacinto // A5 Group 3 // ajr4498

For this week's homework we are going to explore the cross validation testing methodology and applying that to get error estimates on the two algorithms we've used so far:
  - Linear Regression
  - Decision Tree classification
  

In [1]:
# Google colab's default version of scikit-learn isn't the latest, so you will 
# need to update the virtual machine and restart the runtime
!pip install scikit-learn==1.0

     |████████████████████████████████| 23.1 MB 1.5 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [2]:
# import the libraries! If you want to add things here for visualization, please do, 
# but only use sklearn when prompted
import pandas as pd
import numpy as np
from sklearn import tree 
from sklearn.datasets import load_iris
from sklearn.datasets import load_diabetes

# Part 1 - Calculate Generalized Error on Linear Regression with k-fold Cross Validation

## Q1.1 Load in the diabetes data set as a pandas dataframe and series.  
Documentation on the data set is [here](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_diabetes.html).  Name your features dataframe (the independent variables) `df_X` and your target value (the dependent variable) series `s_y`

In [52]:
tup = load_diabetes(return_X_y=True, as_frame=True) # X_y returns a tuple, then as_frame converts each element into a Pandas df
df_X = tup[0]
s_y = tup[1]
df_X.head()


,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068330,-0.092204
2,0.085299,0.050680,0.044451,-0.005671,-0.045599,-0.034194,-0.032356,-0.002592,0.002864,-0.025930
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022692,-0.009362
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031991,-0.046641


## Q1.2 Define a function that creates a linear least squares regression model and a function to predict a continuous value given a linear regression model
The first function should take in two parameters, `df_X`, and `s_y` and return the least squares regression model, $\hat{\beta}$ (using the notation from the ESLII book chapter 3 and HW3).  You can not use any libraries outside of pandas and numpy. Note that the length of beta_hat should be the number of columns in `df_X` + 1. 

The second function should take in two parameters, `beta_hat` - the model generated from the `get_linear_regression` function, and `df_X` - that has the attribute values at which we want to predict a continuous value.  We assume that the format and ordering of `df_X` used to create the model and `df_X` used to generate predictions are consistent. 

In [53]:
def get_linear_regression_model( df_X, s_y ): # NOTE: df_X length = 10
    # first column is the market
    X = df_X # readability
    y = s_y
    # prepend a column of ones for the intercept
    X = np.c_[np.ones(X.shape[0]), X] # .c_ is concatenation on 2nd axis
    # ... whereas .r_ (Rows) is that along 1st axis (rows)

    # matrix algebra to find betaHat
    b = np.linalg.pinv(X.T.dot(X)).dot(X.T).dot(y)

    return b
# confirm correct amount of rows === 11: DONE

In [54]:
# code to check beta_hat
# FIXME: use assert_equal()
np.random.seed(23) 
beta_hat = get_linear_regression_model( pd.DataFrame(np.random.random((34,4))), pd.Series(np.random.random(34)*10.0) )
# length of beta_hat == 5
testdf = pd.DataFrame(np.random.random((3,4))) # random df is 3 rows X 4 col


In [55]:
def predict_linear_regression_value( beta_hat, df_X ):
  # prepend column of 1's to given df_X
  # REMEMBER THIS! Because this matrix method must handle the intercept, hence the prepension
  mod_df_X = np.c_[np.ones(df_X.shape[0]), df_X] # .c_[] a col of 1's to df_X

  return np.matmul(mod_df_X, beta_hat)

In [56]:
predicted_vals = predict_linear_regression_value(beta_hat, testdf)
predicted_vals



array([4.03669521, 4.39502589, 3.90884703])

## Q1.3 Define a function that partitions the dataframe and series data into dictionaries
This function should take in three parameters, `df_X`, `s_y`, and `k`, and returns a tuple of two dictionaries.
In both dictionaries the key is the `k` value (an integer from one to `k` inclusive).
The first dictionary will have the dataframe of the training data that contains approximately $\frac{1}{k}$ of the data (variation due to randomness is acceptable).
The second dictionary will have the series of the target data that contains approximately $\frac{1}{k}$ of the data (variation due to randomness is acceptable). Please note the targets _must match_ the same rows as the dataframe at this index, e.g, the length of the kth partition is the same for the dataframe and series.

Call that function with $k=5$ and create the dictionaries `dict_k_df_X` and `dict_k_s_y`. Print out the number of rows in each fold.  Check that the number of data points in each partition totals the number of data points in the entire dataset. 

Here is some example output from checking the length of the folds:
```
Fold 1 length of dataframe is 88 and length of series is 88
Fold 2 length of dataframe is 96 and length of series is 96
Fold 3 length of dataframe is 88 and length of series is 88
Fold 4 length of dataframe is 79 and length of series is 79
Fold 5 length of dataframe is 91 and length of series is 91
The sum of the number of elements in each fold is 442 and there are 442 rows in the original df
```

In [57]:
# k := key, type int
# output: tuple (dictX, dicty), with each containing 1/k of data
from sklearn.model_selection import KFold

def get_part_indices(df_X, k):
  kf = KFold(n_splits=k)
  folds = kf.split(df_X)
  index_list = []
  for _ in range(k):
    train_index, test_index = next(folds)
    index_list.append(test_index)
  return index_list

def partition_data(df_X, s_y, k): # yields a list of tuples (dict_X, dict_y) of length k
  dict_X = {} # initialize dicts
  dict_y = {}

  keyval = 1 # int from 1 to k
  for test_index in get_part_indices(df_X, k):
    # create the train(X) and target(y) partitions
    partition_X = df_X.iloc[test_index,:]
    partition_y = s_y.iloc[test_index]

    # update dictionary
    dict_X.update({keyval: partition_X})
    dict_y.update({keyval: partition_y})
    keyval+=1 # increment keyval

  return (dict_X, dict_y) # tuple containing dictionaries containing dfs




    



In [58]:
# define list 'all', containing all of the k partitions
(dict_k_df_X, dict_k_s_y) = partition_data(df_X, s_y, 5)


In [59]:
# Check fold sizes
# print out number of rows in each fold

print("Fold 1 length of dataframe is ", len(dict_k_df_X[1].index),\
      " and length of series is ", len(dict_k_s_y[1].index)  )
print("Fold 2 length of dataframe is ", len(dict_k_df_X[2].index),\
      " and length of series is ", len(dict_k_s_y[2].index)  )
print("Fold 3 length of dataframe is ", len(dict_k_df_X[3].index),\
      " and length of series is ", len(dict_k_s_y[3].index)  )
print("Fold 4 length of dataframe is ", len(dict_k_df_X[4].index),\
      " and length of series is ", len(dict_k_s_y[4].index)  )
print("Fold 5 length of dataframe is ", len(dict_k_df_X[5].index),\
      " and length of series is ", len(dict_k_s_y[5].index)  )
print("Sum of aforementioned lengths = 442")








Fold 1 length of dataframe is  89  and length of series is  89
Fold 2 length of dataframe is  89  and length of series is  89
Fold 3 length of dataframe is  88  and length of series is  88
Fold 4 length of dataframe is  88  and length of series is  88
Fold 5 length of dataframe is  88  and length of series is  88
Sum of aforementioned lengths = 442


## Q1.4 Define a function that calculates a regression metric
This function should accept two series of equal length $n$ numpy arrays, `s_y`, and `s_y_hat`. The metric it should calculate is the mean absolute error, $MAE = \sum\limits_{i=1}^n\frac{|{s\_y_i - {s\_y\_hat}_i}|}{n}$ 

Test your function by using the vectors:
```
x = np.array([1,2,3])
y = np.array([2,2,3])
```


In [60]:
def get_mae( s_y, s_y_hat): # assume equal length
  mae = 0
  n = len(s_y)
  for ind in range(n):
    mae += abs(s_y[ind] - s_y_hat[ind])
  return mae/n






In [61]:
# Test it 
x = np.array([1,2,3])
y = np.array([2,2,3]) # :== s_y_hat
get_mae(x,y)



0.3333333333333333

## Q1.5 Calculate the $MAE$ for each fold
For each fold in your dictionaries, calculate the $MAE$.  Use the partition number key as the test set, and all other partitions as the train set. 

Print the min, max, and mean $MAE$ of your 5 folds. 

You must use your helper functions that you wrote above, `get_linear_regression_model`, `predict_linear_regression_value` and `get_mae`. 

In [64]:
mae = np.array([])
r = [1, 2, 3, 4, 5]
for k in dict_k_df_X.keys(): # i.e. in [1, 2, 3, 4, 5]
  r.pop(0)
  # first: 1 TEST, 2 TRAIN
  test = dict_k_s_y[k] # series
  train = dict_k_df_X[k]

  # use helper functions to determine s_y_hat for this k
  betaHat = get_linear_regression_model(train, test)
  values = predict_linear_regression_value(betaHat, train) # np.array type
  
  # need both get_mae parameters to be type np.array
  s_y = np.array(test)
  s_y_hat = np.array(values)

  # calculate M.A.E. for this k value
  mae = np.append(mae, get_mae(s_y, s_y_hat))

array([38.59551547, 41.77984083, 45.14311716, 39.83699235, 42.09152503])

In [63]:
print("The min MAE is {:.2f}, the max MAE is {:.2f}, and the mean MAE is {:.2f}".format(mae.min(),mae.max(),mae.mean()))

The min MAE is 38.60, the max MAE is 45.14, and the mean MAE is 41.49


# Part 2 - Find the best hyperparameter to use in a Decision Tree 

## Q2.1 Load the iris data in as a pandas dataframe and a series
Documentation on the data set is [here](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html). Name your features dataframe (the independent variables) `df_X` and your class label (the dependent variable) series `s_y`

In [15]:
(df_X, s_y) = load_iris(return_X_y=True, as_frame=True) # 150 rows each
df_X

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


## Q2.2 Partition `df_X` and `s_y` into $5$ partitions of roughly equal size
Make 2 dictionaries, with the key of each dictionary the fold number.  The value of the dictionary `dict_k_df_X` is the $k^{th}$ partition of the data, and the value of the dictionary `dict_k_s_y` is the corresponding $k^{th}$ target classification.  Print out the number of rows in each fold.  Check that the number of data points in each partition totals the number of data points in the entire dataset. 

Note, you can reuse the functions from Section 1. 

In [16]:
from sklearn.model_selection import KFold

arr_X = df_X.to_numpy()
arr_y = s_y.to_numpy()
kfolds = KFold(n_splits=5)

dict_k_df_X = {}
dict_k_s_y = {}
df_X_indices = []
s_y_indices = []
part = -1

for train, test in kfolds.split(arr_X):
  part += 1
  df_X_indices.append(test)

part = -1
for train, test in kfolds.split(arr_y):
  part += 1
  s_y_indices.append(test)

for idx, lst in enumerate(df_X_indices):
  dict_k_df_X[idx] = df_X[lst[0]:(lst[-1]+1)]
for idx, lst in enumerate(s_y_indices):
  dict_k_s_y[idx] = s_y[lst[0]:(lst[-1]+1)]

In [17]:
# input: X db, y Series, k
# output: tuple (db_X, s_y) with exclusion applied
def df_exclude_k(df_X, s_y, k):
    excluded_df = pd.DataFrame(dict_k_df_X[k])
    excluded_s = pd.Series(dict_k_s_y[k])

    df_y = s_y.to_frame() # series into df
    df_new = df_X.merge(excluded_df, how='left', indicator=True)
    df_new = df_new[df_new['_merge'] == 'left_only']
    df_new = df_new.drop(columns=['_merge'])

    start_ind = excluded_s.index[0]
    stop_ind = excluded_s.index[-1]

    s_new_1 = s_y.iloc[0:start_ind]
    s_new_2 = s_y.iloc[stop_ind+1:]
    
    s_new = s_new_1.combine_first(s_new_2)

    if k > 2: # FIXME: length mismatch at k == 3 || 4
        s_new = s_new.iloc[:-1]

    return df_new, s_new # X, y format

df_exclude_k(df_X, s_y, 3)


(     sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
 0                  5.1               3.5                1.4               0.2
 1                  4.9               3.0                1.4               0.2
 2                  4.7               3.2                1.3               0.2
 3                  4.6               3.1                1.5               0.2
 4                  5.0               3.6                1.4               0.2
 ..                 ...               ...                ...               ...
 145                6.7               3.0                5.2               2.3
 146                6.3               2.5                5.0               1.9
 147                6.5               3.0                5.2               2.0
 148                6.2               3.4                5.4               2.3
 149                5.9               3.0                5.1               1.8
 
 [119 rows x 4 columns], 0      0.0
 1      0.0
 2

## Q2.3 Define a function that calculates accuracy
The function should accept two series and compare each element for equality.  The accuracy is the number of equal elements divided by the total number of elements.

Test your accuracy function by calling it with the `s_y` loaded from the iris data set and an array of the same length containing all $1$ values. 

In [18]:
# create a function that takes dictionary and current fold,
# and returns a dictionary with all OTHER folds
def merge_folds(dict_X, dict_y, curr_k):
    del dict_X[curr_k]
    del dict_y[curr_k]
    return dict_X, dict_y

In [19]:
def get_acc( s_1, s_2 ): # assume same Series length
  num_equal = 0
  total_elements = s_1.size
  for e1 in s_1:
    for e2 in s_2:
      if e2 == e1:
        num_equal += 1
        break # stop comparing to not count e1 element multiple times

  return num_equal/total_elements

In [20]:
get_acc(s_y,np.ones(len(s_y)))

0.3333333333333333

## Q2.4 Using Nested Cross validation, find the best hyperparameter
Use the [Decision Tree Classifier](https://scikit-learn.org/stable/modules/tree.html#classification) class to build a decision tree inside of a 5-fold cross validation loop.  The partitions you already created in 2.2 will be the outer loop.  In the inside loop you should use 4-fold cross validation (so you don't have to partition _again_) to find the best value for `min_impurity_decrease`.  Use the Gini Index as your impurity measure. 
    Calculate the mean accuracy across the 4 folds of your inner loop for all the candidate `min_impurity_decrease` values, and print the value.  Use the array `np.array([0.1,0.25,0.3,0.4])` as the candidates for the best hyperparameter. If there is a tie (two `min_impurity_decrease` values give the same highest accuracy), choose the lowest `min_impurity_decrease` value. 

For each inner loop, select the best `min_impurity_decrease` and train the outer fold training data on using that value. 

For each of the 5 executions of the inner loop, your output should look something like this:
```
Testing 0.10 min impurity decrease
	Average accuracy over 4 folds is 0.95
Testing 0.25 min impurity decrease
	Average accuracy over 4 folds is 0.86
Testing 0.30 min impurity decrease
	Average accuracy over 4 folds is 0.63
Testing 0.40 min impurity decrease
	Average accuracy over 4 folds is 0.27

Best min impurity decrease is 0.1

```

In [21]:
# import libraries from SKLearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

def run_cross_validation_on_trees(X, y, min_impurity_decrease, cv=4, scoring='accuracy'):
    cv_scores_list = []
    cv_scores_std = []
    cv_scores_mean = []
    accuracy_scores = []
    #for depth in tree_depths:
    tree_model = DecisionTreeClassifier(min_impurity_decrease=pos_min_impurity) # default impurity = GINI
    cv_scores = cross_val_score(tree_model, X, y, cv=cv, scoring=scoring)

    cv_scores_list.append(cv_scores)
    cv_scores_mean.append(cv_scores.mean())
    cv_scores_std.append(cv_scores.std())

    accuracy_scores.append(tree_model.fit(X, y).score(X, y))
    cv_scores_mean = np.array(cv_scores_mean)
    cv_scores_std = np.array(cv_scores_std)
    accuracy_scores = np.array(accuracy_scores)

    return cv_scores_list, cv_scores_mean, accuracy_scores # ORDER



In [37]:
possible_min_impurity_decrease = np.array([0.1,0.25,0.3,0.4]) # the answer is one of these
# Outer loop
outer_acc = np.array([])
for k in dict_k_df_X.keys():

    k_test_y = dict_k_s_y[k] # partition 0 y
    k_inner_X, k_inner_y = df_exclude_k(df_X, s_y, k) # partition 1-4 X, y

    # print("k = ", k, " List, Means, and Accuracies:")
    
    # HAVE: outer testing y, raw inner TRAINING data
    for pos_min_impurity in possible_min_impurity_decrease:
        # print(pos_min_impurity, ": ")
        cv_list, cv_means, accuracy_scores = run_cross_validation_on_trees(k_inner_X, k_inner_y, pos_min_impurity) # use defaults
        # print(accuracy_scores)
        # need to keep track of average accuracy for each of pos_min_imp levels
        outer_acc = np.append(outer_acc, accuracy_scores)

ind01 = list(range(0, 17, 4))
ind025 = list(range(1, 17, 4))
ind03 = list(range(2, 17, 4))
ind04 = list(range(3, 17, 4))
sum01, sum025, sum03, sum04 = 0, 0, 0, 0

for idx, avg_acc in enumerate(outer_acc):
    if idx in ind01:
      sum01 += avg_acc
    elif idx in ind025:
      sum025 += avg_acc
    elif idx in ind03:
      sum03 += avg_acc
    elif idx in ind04:
      sum04 += avg_acc
    
avg01 = sum01 / 5
avg025 = sum025 / 5
avg03 = sum03 / 5
avg04 = sum04 / 5

print('Testing 0.10 min impurity decrease')
print('\tAverage accuracy over 4 folds is {:.2f}'.format(avg01))
print('Testing 0.25 min impurity decrease')
print('\tAverage accuracy over 4 folds is {:.2f}'.format(avg025))
print('Testing 0.30 min impurity decrease')
print('\tAverage accuracy over 4 folds is {:.2f}'.format(avg03))
print('Testing 0.40 min impurity decrease')
print('\tAverage accuracy over 4 folds is {:.2f}'.format(avg04))

avgs = {.10: avg01,
        .25: avg025,
        .30: avg03,
        .40: avg04
        }
best_min_impurity = max(avgs, key=avgs.get)
print('\nBest min impurity decrease is {:.2f}'.format(best_min_impurity))


# now use np.mean(sum??) for mean, and determine the minimum of those values



        
    # Use best min impurity decrease to train model
    
    # outer accuracy calculation 

    #outer_acc = np.append(outer_acc,this_acc) # make sure and calculate this_acc in your loop

Testing 0.10 min impurity decrease
	Average accuracy over 4 folds is 0.96
Testing 0.25 min impurity decrease
	Average accuracy over 4 folds is 0.66
Testing 0.30 min impurity decrease
	Average accuracy over 4 folds is 0.56
Testing 0.40 min impurity decrease
	Average accuracy over 4 folds is 0.33

Best min impurity decrease is 0.10


## Q2.5 Show the generalized performance of the classifier 
Show the generalized performance of the classifier by printing the min, max, and mean accuracy of the outer fold test sets.

In [44]:
print('Minimum Accuracy of Outer Fold Test Set: ', min(outer_acc))
print('Maximum Accuracy of Outer Fold Test Set: ', max(outer_acc))
print('Mean Accuracy of Outer Fold Test Set: ', outer_acc.mean())

Minimum Accuracy of Outer Fold Test Set:  0.4166666666666667
Maximum Accuracy of Outer Fold Test Set:  0.9833333333333333
Mean Accuracy of Outer Fold Test Set:  0.735406162464986
